<h1 style="text-align:center;">Test ESKM Connection</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook will contain script for testing connection to the ESKM (Enterprise Secure Key Manager) system for HPE iLO systems. This code is written in Python and uses the redfish library to perform the required operations.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
        <li>getpass - to take password as inputs securely</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is our test_ESKM_connection function overview.</h3><br>
    
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to do this.<br>

1. The <b>test_ESKM_connection_ilo_firmware</b> function takes one parameter - <b>_redfishobj</b>.
    <br>
    
2. Initialize the <b>eskm_uri</b> and <b>body</b> variables to <b>None</b> and an empty dictionary, respectively.
    <br>

3. Call the <b>get_resource_directory()</b> function by passing <b>_redfishobj</b> as a parameter to retrieve the resource directory of the server. If the resource directory is not available or disabled, find the relevant URI by navigating through the server's API.
    <br>

4. If the resource directory is available, find the relevant URI from the resource directory.
    <br>

5. Check if <b>eskm_uri</b> has been set. If <b>eskm_uri</b> is not <b>None</b>, retrieve the URI for the ESKM service.
    <br>

6. If the ESKM URI is available, retrieve the URI for testing the ESKM connection from the response.
    <br>

7. Set the <b>Action</b> field of the <b>body</b> dictionary to <b>"HpeESKM.TestESKMConnections"</b>.
    <br>
    
8. Call the <b>post()</b> method of the <b>_redfishobj</b> object with the ESKM test connection URI and the <b>body</b> dictionary as parameters.
    <br>
    
9. If the HTTP response status code is not 200 or 201, print an error message. If the HTTP response status code is 400, print the iLO extended info error message. If the HTTP response status code is 200 or 201, print a success message and the JSON-formatted response.
    
    
10. If eskm_uri is None, print an error message.
    <br>
</div>

In [ ]:
def test_ESKM_connection(_redfishobj):

    eskm_uri = None
    body = dict()

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        security_service_uri = managers_members_response.obj.Oem.Hpe.Links['SecurityService']['@odata.id']
        security_service_response = _redfishobj.get(security_service_uri)
        eskm_uri = security_service_response.obj.Links['ESKM']['@odata.id']

    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeESKM.' in instance['@odata.type']:
                eskm_uri = instance['@odata.id']
                break

    if eskm_uri:
        eskm_test_conn_uri = _redfishobj.get(eskm_uri).obj['Actions']['#HpeESKM.TestESKMConnections']['target']
        body["Action"] = "HpeESKM.TestESKMConnections"
        resp = _redfishobj.post(eskm_test_conn_uri, body)
        #If iLO responds with something outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.</h3>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to test connection to ESKM Systems.

1. The rest opertions are handled by redfish object which is created by RedfishClient<br>
<br>
    
2. <b>RedfishClient</b> class takes 3 parameters<i> iLO url, Username, Password</i> and returns a redfish object.<br>
    <br>

3. Once you enter everything a redfish obejct is created with the help of RedfishClient class.<br>
    <br>

4. Now we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful you are logged in, But if           Server       is not avaialbe we get an error saying "server unreachable"<br>
    <br>
    
5. Then we call <b>test_ESKM_connection</b> with our redfish object as parameter.<br>  <br>  

6. REDFISHOBJ.logout() will take care of logging you out of the ilo.<br><br>

</div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b>use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    test_ESKM_connection(REDFISHOBJ)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help please check the follwoing links for reference:
    <br>
    1. Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">Python ilorest library</a><br>
    2. HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">ilorestful api explorer</a><br>
    3. HPE iLO Redfish Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/">HPE iLO Redfish API Documentation</a><br>
    4. REST API: <a href = "https://restfulapi.net/">Restful API</a><br>    
</div>